In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import swifter
from scipy.stats import norm

In [2]:
df2020 = pd.read_excel('Dataset2020.xlsx', sheet_name='2020')
print(df2020.head())
dfYieldCurves = pd.read_excel('Dataset2020.xlsx', sheet_name='YieldCurves', index_col='Date')
print(dfYieldCurves.head())

   id PurchaseDate  PriceAtCoupon0(%)  CurrentCoupon  CouponsRemaining  \
0   1   1994-09-07             62.489            311               327   
1   2   1994-09-07             97.044            311               359   
2   3   1994-09-07             75.515             25                19   
3   4   1994-09-07             69.743             25                43   
4   5   1994-09-07             64.350             25                31   

  PaymentPeriod  BondsHeld  YearlyRate  FaceValue Corporate  
0       Monthly         26    0.135090     157000       Yes  
1       Monthly          9    0.160691     125000       Yes  
2      Annually         10    0.190721      99000       Yes  
3      Annually        590    0.092318     103000       Yes  
4      Annually         71    0.121696      90000       Yes  
            Nr     025YR     050YR     075YR     100YR     125YR     150YR  \
Date                                                                         
1991-01-02   1  0.109902  0

In [8]:
def discount_rate(payment_type, M, time = '2020-08-26'):
    
    rate_raw = dfYieldCurves.loc[time, :].iloc[1:].values
    
    if payment_type == 'Monthly':
        rate = np.repeat(rate_raw[-1]/12, M + 1)
        for i in range(0, min(len(rate)-1, len(rate_raw)*3)):
            rate[i+1] = rate_raw[i//3]/12
        rate[0] = 0
    elif payment_type == 'Annually':
        rate = np.repeat(rate_raw[-1], M + 1)
        for i in range(0, min(len(rate)-1, len(rate_raw)//4)):
            rate[i+1] = rate_raw[i*4]
        rate[0] = 0
    else:
        rate = np.repeat(rate_raw[-1]/4, M + 1)
        for i in range(0, min(len(rate)-1, len(rate_raw))):
            rate[i+1] = rate_raw[i]/4
        rate[0] = 0
        
    return rate

def net_present_value(Porig, spread, rate, c, FV, M, payment_type):
    
    if payment_type == 'Monthly':
        k=12
    elif payment_type == 'Annually':
        k=1
    else:
        k=4
    
    c=c/k
    
    payment_series = np.repeat(c * FV, M + 1)
    payment_series[0] = -1 * Porig
    payment_series[M] += FV
    
    discount_factor = np.array([1/((1+r+spread)**i) for i, r in enumerate(rate)])

    npv = np.dot(payment_series, discount_factor)
    
    return npv



def CouponTime(pd, cc, cr, payment_type):
    
    if payment_type == 'Monthly':
        k=1
    elif payment_type == 'Annually':
        k=12
    else:
        k=3
    
    t = pd
    t_m = np.datetime64(t, 'M')
    t_d = t - t_m
    date_coupon_before = t_m + np.timedelta64(cc*k,'M')
    data_coupon_after = t_m + np.timedelta64(cc*k+k,'M')
    date_coupon_before = date_coupon_before + t_d
    data_coupon_after = data_coupon_after + t_d
    
    d0 = int((np.datetime64('2020-08-26') - date_coupon_before).days)
    d1 = int((data_coupon_after - np.datetime64('2020-08-26')).days)
   
    return (d0/(d0+d1), (d1/(d0+d1)+cr)*k/12)

def PriceClean(DirtyPrice, time, cr, FV, payment_type):
    
    if payment_type == 'Monthly':
        k=12
    elif payment_type == 'Annually':
        k=1
    else:
        k=4
        
    p = DirtyPrice - time*cr/k*FV
    
    return p

In [9]:
def SpreadYr(spread, payment_type):
    
    if payment_type == 'Monthly':
        k=12
    elif payment_type == 'Annually':
        k=1
    else:
        k=4
        
    return spread*k

def PriceNow(p, FV):
    
    return p/FV*100



def PD(spreadyr):
    
    if spreadyr<s1[0]:
        pd=0.001
    elif s1[0]<=spreadyr<s1[1]:
        pd=0.01
    elif s1[1]<=spreadyr<s1[2]:
        pd=0.12
    elif s1[2]<=spreadyr<s1[3]:
        pd=0.36
    elif s1[3]<=spreadyr<s1[4]:
        pd=0.52
    elif s1[4]<=spreadyr<s1[5]:
        pd=0.75
    elif s1[5]<=spreadyr:
        pd=0.85
    else:
        pd=1
        
    return pd

def capital_requirement_cands(PD, M):
  
    R =  0.12 * ( (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) )
    R += 0.24 * (1 - ( (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) ) )
    K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
    K *= 0.75
    b = (0.11852-0.05478*np.log(PD))**2
    K = K*(1+(M-2.5)*b)/(1-1.5*b)

    return K

In [10]:
def Provision(PD, EAD, N):
    
    return 0.75*PD*EAD*N

def CapitalRequirement(K, EAD, N):
    
    return 0.115*K*EAD*N

In [11]:
def func(row):
    sp = fsolve(lambda x : net_present_value(spread = x, Porig=row['PriceAtCoupon0(%)']/100*row['FaceValue'], rate=discount_rate(row['PaymentPeriod'], row['CurrentCoupon']+row['CouponsRemaining'], row['PurchaseDate']), c=row['YearlyRate'], FV=row['FaceValue'], M=row['CurrentCoupon']+row['CouponsRemaining'], payment_type=row['PaymentPeriod']), x0 = 0, xtol=1.49012e-8)[0]
    npv = net_present_value(Porig=row['PriceAtCoupon0(%)']/100*row['FaceValue'], spread=sp, rate=discount_rate(row['PaymentPeriod'], row['CurrentCoupon']+row['CouponsRemaining']), c=row['YearlyRate'], FV=row['FaceValue'], M=row['CurrentCoupon']+row['CouponsRemaining'], payment_type=row['PaymentPeriod'])
    timeb, timea = CouponTime(row['PurchaseDate'], row['CurrentCoupon'], row['CouponsRemaining'], row['PaymentPeriod'])
    pc = PriceClean(npv, timeb, row['YearlyRate'], row['FaceValue'], row['PaymentPeriod'])
    spyr = SpreadYr(sp, row['PaymentPeriod'])
    npvd = PriceNow(npv, row['FaceValue'])
    npvc = PriceNow(pc, row['FaceValue'])
    return pd.Series([sp, spyr, npvd, npvc, timea])

df2020[['Spread', 'SpreadYearly','PriceDirty', 'PriceClean', 'M']] = df2020.swifter.apply(func, axis = 1)
print(df2020.head(20))

s = df2020.SpreadYearly.values
s1 = [np.quantile(s, 0.05), np.quantile(s, 0.2), np.quantile(s, 0.4), np.quantile(s, 0.6), np.quantile(s, 0.75), np.quantile(s, 0.9)]


    id PurchaseDate  PriceAtCoupon0(%)  CurrentCoupon  CouponsRemaining  \
0    1   1994-09-07             62.489            311               327   
1    2   1994-09-07             97.044            311               359   
2    3   1994-09-07             75.515             25                19   
3    4   1994-09-07             69.743             25                43   
4    5   1994-09-07             64.350             25                31   
5    6   1994-09-07             71.000            311               194   
6    7   1994-09-07             69.023             25                28   
7    8   1994-09-07             70.210            311               146   
8    9   1994-09-07             73.500            311               521   
9   10   1994-09-07             70.700            311               224   
10  11   1994-09-07             73.420            103                10   
11  12   1994-09-07             76.755            103                10   
12  13   1994-09-07     

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
def func(row):
    Pd = PD(row['SpreadYearly'])
    k = capital_requirement_cands(Pd, row['M'])
    provision = Provision(Pd, row['PriceClean'], row['BondsHeld'])
    capitalr = CapitalRequirement(k, row['PriceClean'], row['BondsHeld'])
    return pd.Series([Pd, k, provision, capitalr])

df2020[['PD', 'K', 'Provisions', 'CapitalRequirements']] = df2020.swifter.apply(func, axis = 1)
print(df2020.head(20))


    id PurchaseDate  PriceAtCoupon0(%)  CurrentCoupon  CouponsRemaining  \
0    1   1994-09-07             62.489            311               327   
1    2   1994-09-07             97.044            311               359   
2    3   1994-09-07             75.515             25                19   
3    4   1994-09-07             69.743             25                43   
4    5   1994-09-07             64.350             25                31   
5    6   1994-09-07             71.000            311               194   
6    7   1994-09-07             69.023             25                28   
7    8   1994-09-07             70.210            311               146   
8    9   1994-09-07             73.500            311               521   
9   10   1994-09-07             70.700            311               224   
10  11   1994-09-07             73.420            103                10   
11  12   1994-09-07             76.755            103                10   
12  13   1994-09-07     

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
print(np.sum(df2020.Provisions))
print(np.sum(df2020.CapitalRequirements))

135174.6837755015
56428.05969453512


In [14]:
time=df2020.PurchaseDate[0]
rate_raw = dfYieldCurves.loc[time, :].iloc[1:].values
print(rate_raw)
rate_new = dfYieldCurves.loc['2020-08-26', :].iloc[1:].values
print(rate_new)

print(s1)

[0.0565071  0.06224712 0.06693654 0.07059234 0.07342921 0.07563879
 0.07737299 0.07874868 0.07985466 0.08075795 0.08150884 0.08214497
 0.08269434 0.0831778  0.08361077 0.08400469 0.08436802 0.084707
 0.0850263  0.08532936 0.08561877 0.08589646 0.08616388 0.08642214
 0.08667206 0.08691426 0.08714922 0.08737725 0.08759859 0.08781339
 0.08802175 0.08822368 0.08841917 0.08860819 0.08879065 0.08896646
 0.08913552 0.08929773 0.08945298 0.08960115 0.08974217 0.08987595
 0.09000242 0.09012155 0.09023329 0.09033767 0.09043468 0.09052439
 0.09060685 0.09068217 0.09075045 0.09081184 0.09086649 0.09091458
 0.09095631 0.09099189 0.09102153 0.09104549 0.09106401 0.09107735
 0.09108577 0.09108954 0.09108893 0.09108421 0.09107567 0.09106356
 0.09104816 0.09102973 0.09100854 0.09098483 0.09095886 0.09093085
 0.09090104 0.09086965 0.09083689 0.09080295 0.09076804 0.09073233
 0.09069599 0.09065917 0.09062204 0.09058473 0.09054736 0.09051006
 0.09047294 0.09043609 0.09039962 0.09036359 0.0903281  0.090293